In [1]:
import numpy as np
import cv2 as cv
import sys

# [1] YOLOv3 모델과 클래스 이름 구성 함수
def construct_yolo_v3():
    # coco_names.txt 파일에서 COCO 데이터셋의 클래스 이름 불러오기
    f = open('coco_names.txt', 'r')
    class_names = [line.strip() for line in f.readlines()]  # 각 줄의 개행문자 제거
    
    # YOLO 모델 로드 (가중치와 설정 파일)
    model = cv.dnn.readNet('yolov3.weights', 'yolov3.cfg')
    
    # 모델의 모든 레이어 이름 가져오기
    layer_names = model.getLayerNames()
    
    # 출력 레이어만 선택 (YOLOv3에서 탐지 결과가 나오는 레이어)
    out_layers = [layer_names[i - 1] for i in model.getUnconnectedOutLayers()]
    
    return model, out_layers, class_names

# [2] YOLO로 이미지에서 객체 탐지하는 함수
def yolo_detect(img, yolo_model, out_layers):
    # 이미지 높이와 너비 가져오기
    height, width = img.shape[0], img.shape[1]

    # 입력 이미지 전처리: blob 만들기
    # scale factor 1/256, 크기 448x448으로 resize, BGR->RGB 변환
    test_img = cv.dnn.blobFromImage(
        img,         # 원본 이미지
        1.0/256,     # 스케일 팩터 (픽셀값을 0~1로)
        (448,448),   # 크기 리사이즈
        (0,0,0),     # Mean 값 (평균값 빼기)
        swapRB=True  # BGR → RGB 변환
    )

    # YOLO 모델에 blob 입력
    yolo_model.setInput(test_img)

    # forward()로 출력 레이어 실행해서 탐지 결과 받기
    output3 = yolo_model.forward(out_layers)

    # 박스, 신뢰도, 클래스 ID 리스트 초기화
    box, conf, id = [], [], []

    # output3에는 여러 출력 레이어별 탐지 결과가 들어있음
    for output in output3:
        for vec85 in output:
            scores = vec85[5:]            # 앞 5개는 위치, 뒤는 클래스별 점수
            class_id = np.argmax(scores)  # 가장 높은 클래스 선택
            confidence = scores[class_id] # 그 클래스의 신뢰도
            if confidence > 0.5:          # 신뢰도 50% 이상인 것만 사용
                centerx, centery = int(vec85[0]*width), int(vec85[1]*height)  # 중심 좌표
                w, h = int(vec85[2]*width), int(vec85[3]*height)              # 박스 너비, 높이
                x, y = int(centerx - w/2), int(centery - h/2)                  # 좌상단 좌표로 변환
                box.append([x, y, x+w, y+h])      # [x1,y1,x2,y2]
                conf.append(float(confidence))    # 신뢰도 저장
                id.append(class_id)               # 클래스 ID 저장

    # NMS (Non-Maximum Suppression)로 겹치는 박스 제거
    ind = cv.dnn.NMSBoxes(box, conf, 0.5, 0.4)

    # 최종 물체 리스트: 박스 좌표 + 신뢰도 + 클래스ID
    objects = [box[i] + [conf[i]] + [id[i]] for i in range(len(box)) if i in ind]

    return objects

# [3] YOLOv3 모델과 클래스 이름 구성하기
model, out_layers, class_names = construct_yolo_v3()

# [4] 클래스별 랜덤 색상 생성 (바운딩 박스 색상)
colors = np.random.uniform(0, 255, size=(len(class_names), 3))

# [5] 비디오 캡처 장치 열기 (웹캠 사용)
cap = cv.VideoCapture(0, cv.CAP_DSHOW)  # Windows는 CAP_DSHOW로 안정성 높임
if not cap.isOpened():
    sys.exit('카메라 연결 실패')

import time

# [6] 처리 시간과 FPS 계산 위한 변수 초기화
start = time.time()
n_frame = 0

# [7] 메인 루프: 실시간 프레임 읽고 탐지 수행
while True:
    ret, frame = cap.read()  # 프레임 읽기
    if not ret:
        sys.exit('프레임 획득에 실패하여 루프를 나갑니다.')
        
    # YOLO로 현재 프레임 탐지 수행
    res = yolo_detect(frame, model, out_layers)   
 
    # 탐지된 물체에 바운딩 박스와 텍스트 표시
    for i in range(len(res)):
        x1, y1, x2, y2, confidence, id = res[i]
        text = str(class_names[id]) + ' %.3f' % confidence  # 클래스 + 신뢰도
        cv.rectangle(frame, (x1, y1), (x2, y2), colors[id], 2)
        cv.putText(frame, text, (x1, y1 + 30), cv.FONT_HERSHEY_PLAIN, 1.5, colors[id], 2)
    
    # 결과 프레임 표시
    cv.imshow("Object detection from video by YOLO v.3", frame)
    
    n_frame += 1  # 프레임 카운트 증가

    key = cv.waitKey(1)  # 1ms 대기 (빠른 프레임 처리)
    if key == ord('q'):  # 'q' 키를 누르면 루프 종료
        break

# [8] 루프 종료 후 FPS와 처리 시간 출력
end = time.time()
print('처리한 프레임 수 =', n_frame)
print('경과 시간 =', end - start)
print('초당 프레임 수 (FPS) =', n_frame / (end - start))

# [9] 리소스 정리
cap.release()          # 카메라 연결 해제
cv.destroyAllWindows() # 모든 OpenCV 창 닫기

처리한 프레임 수 = 131
경과 시간 = 25.359620094299316
초당 프레임 수 (FPS) = 5.165692526657684
